# Day 12: Rain Risk

https://adventofcode.com/2020/day/12

## Part 1

For this question, part of me wants to design a `Ferry` class that can "drive" itself based on a direction provided. Another part of me wants to process the directions in a bulk action.

Given the rules, the directional movements - `N`, `S`, `E`, `W` - are independent of the ship's facing. So those could all be compiled first to give a new starting point. After that, the turns (`R` and `L`) and forward movements (`F`) can be calculated sequentially to adjust the new origin point and arrive at the destination.

Of course, without seeing part 2 yet, we could very well have to change up that algorithm so it uses the original methodology. And making sequential changes to the ship position just processing all the rules in order takes just about as long as trying to bulk change them.

So, no need to get too smart: just build the `Ferry`, set sail, see where we end up.

In [51]:
from pathlib import Path

INPUTS = Path('input.txt').resolve().read_text().strip()
DIRECTIONS = INPUTS.split('\n')

In [52]:
from typing import List

class Ferry:
    """Representation of the ferry from the problem.

    Records its position as X and Y coordinates. These translate to cardinal directions:

    - positive Y is "North"
    - negative Y is "South"
    - positive X is "East"
    - negative X is "West"

    Records its heading as a constant, matching 'N', 'E', 'S', or 'W'.

    Can be given a direction using `follow_direction`, taking one line from the day's input and applying its changes
    to the position and heading as needed.

    Finally, use property `manhattan_distance` to calculate the current Manhattan distance
    from the origin point (0, 0). This distance is the sum of the absolute values of its X and Y coordinates.
    """

    HEADING_NORTH = 'N'  # Nobody
    HEADING_EAST = 'E'   # Enjoys
    HEADING_SOUTH = 'S'  # Soviet
    HEADING_WEST = 'W'   # Womble

    # Headings are defined in clockwise order on the compass.
    # A LEFT turn is a NEGATIVE change in index,
    # and a RIGHT turn is a POSITIVE change in index.
    HEADINGS = [
        HEADING_NORTH,
        HEADING_EAST,
        HEADING_SOUTH,
        HEADING_WEST,
    ]
    TURN_LEFT = 'L'
    TURN_RIGHT = 'R'
    MOVE_FORWARD = 'F'

    def __init__(
        self,
        heading: str = HEADING_EAST,
        pos_x: int = 0,
        pos_y: int = 0,
    ):
        self.heading = heading
        self.pos_x = pos_x
        self.pos_y = pos_y

    def follow_directions(self, directions: List[str]):
        """Accepts a bulk set of directions to follow, and runs then sequentially."""
        for direction in directions:
            self.follow_direction(direction)

    def follow_direction(self, direction: str):
        """Follows the instructions of a single direction, making changes to
        the ship position and heading.
        """
        command = direction[0]
        value = int(direction[1:])
        if command in self.HEADINGS:
            self.go_that_way(command, value)
        elif command == self.MOVE_FORWARD:
            self.go_forward(value)
        elif command in (self.TURN_LEFT, self.TURN_RIGHT):
            self.turn(command, value)
        else:
            raise ValueError("Command %s not recognized (tried `%s` and `%s`)" % (direction, command, value))

    def go_that_way(self, direction: str, distance: int):
        """Moves the boat that-a-way a set distance."""
        # For lack of a switch statement in Python,
        # we use dictionaries, instead
        funcs = {
            self.HEADING_NORTH: self.go_north,
            self.HEADING_EAST: self.go_east,
            self.HEADING_SOUTH: self.go_south,
            self.HEADING_WEST: self.go_west,
        }
        go_func = funcs[direction]
        go_func(distance)

    def go_north(self, distance: int):
        """Positive Y change"""
        self.pos_y += distance

    def go_east(self, distance: int):
        """Positive X change"""
        self.pos_x += distance

    def go_south(self, distance: int):
        """Negative Y change"""
        self.pos_y -= distance

    def go_west(self, distance: int):
        """Negative X change"""
        self.pos_x -= distance

    def go_forward(self, distance: int):
        """Basically a `go_that_way` for the direction we're facing."""
        self.go_that_way(self.heading, distance)

    def turn(self, direction: str, angle: int):
        """Changes the heading of the ship by a set angle."""
        curr_heading_idx = self.HEADINGS.index(self.heading)
        num_turns = angle // 90

        # self.HEADINGS is organized so that
        # a turn to the LEFT means a negative change in the index,
        # whereas turning RIGHT means a positive change.
        if direction == self.TURN_LEFT:
            heading_change = curr_heading_idx - num_turns
        else:
            heading_change = curr_heading_idx + num_turns
        new_heading_idx = heading_change % 4
        self.heading = self.HEADINGS[new_heading_idx]

    @property
    def manhattan_distance(self):
        return abs(self.pos_x) + abs(self.pos_y)

In [53]:
the_ferry = Ferry()
the_ferry.follow_directions(DIRECTIONS)

print("At the end of the directions,")
print(f"the ferry is facing {the_ferry.heading} at position ({the_ferry.pos_x}, {the_ferry.pos_y}),")
print(f"with a Manhattan distance of {the_ferry.manhattan_distance}.")

At the end of the directions,
the ferry is facing W at position (-163, -218),
with a Manhattan distance of 381.


## Part 2

Well, this changes makes things a bit complicated, for sure.

So we need to track both the ship's position and that of a waypoint the ship is headed toward.

Seems we need a new class that defines these rules.

In [54]:
from typing import List

class FerryPartDeux:
    # I considered inheriting from Ferry to re-use some of that tooling,
    # but I felt they're too different from each other to really benefit from that
    # without creating some weird conditions in which the rules of Part 1 started applying
    # to part 2.
    # Whatever, build it again!

    HEADING_NORTH = 'N'
    HEADING_EAST = 'E'
    HEADING_SOUTH = 'S'
    HEADING_WEST = 'W'
    # This time the order of the headings doesn't matter,
    # as they're just used for convenience in `follow_direction`'s conditions.
    HEADINGS = [
        HEADING_NORTH,
        HEADING_EAST,
        HEADING_SOUTH,
        HEADING_WEST,
    ]
    ROTATE_COUNTERCLOCKWISE = 'L'
    ROTATE_CLOCKWISE = 'R'
    ROTATIONS = [
        ROTATE_COUNTERCLOCKWISE,
        ROTATE_CLOCKWISE,
    ]
    MOVE_TO_WAYPOINT = 'F'

    def __init__(
        self,
        pos_x: int = 0,
        pos_y: int = 0,
        waypoint_x: int = 10,
        waypoint_y: int = 1,
    ):
        self.pos_x = pos_x
        self.pos_y = pos_y
        self.waypoint_x = waypoint_x
        self.waypoint_y = waypoint_y

    def follow_directions(self, directions: List[str]):
        """Accepts a bulk set of directions to follow, and runs then sequentially."""
        for direction in directions:
            self.follow_direction(direction)

    def follow_direction(self, direction: str):
        """Follows the instructions of a single direction, making changes to
        the waypoint and ship positions.
        """
        command = direction[0]
        value = int(direction[1:])
        if command in self.HEADINGS:
            self.move_waypoint_that_way(command, value)
        elif command in self.ROTATIONS:
            self.rotate_waypoint(command, value)
        elif command == self.MOVE_TO_WAYPOINT:
            self.go_to_waypoint(value)
        else:
            raise ValueError("Command %s not recognized (tried `%s` and `%s`)" % (direction, command, value))

    def move_waypoint_that_way(self, direction: str, distance: int):
        funcs = {
            self.HEADING_NORTH: self.move_waypoint_north,
            self.HEADING_EAST: self.move_waypoint_east,
            self.HEADING_SOUTH: self.move_waypoint_south,
            self.HEADING_WEST: self.move_waypoint_west,
        }
        move_func = funcs[direction]
        move_func(distance)

    def move_waypoint_north(self, distance: int):
        """POSITIVE waypoint Y change."""
        self.waypoint_y += distance

    def move_waypoint_east(self, distance: int):
        """POSITIVE waypoint X change."""
        self.waypoint_x += distance

    def move_waypoint_south(self, distance: int):
        """NEGATIVE waypoint Y change."""
        self.waypoint_y -= distance

    def move_waypoint_west(self, distance: int):
        """NEGATIVE waypoint X change."""
        self.waypoint_x -= distance

    def rotate_waypoint_clockwise(self):
        """Rotates the waypoint clockwise around the ship 90 degrees."""
        # Back to high school math on this one.
        # Rotating 90 degrees clockwise around the origin point
        # means translating (x, y) to (y, -x).
        # As we treat the ship as the waypoint's origin, that's a piece of cake.
        self.waypoint_x, self.waypoint_y = self.waypoint_y, -self.waypoint_x

    def rotate_waypoint_counterclockwise(self):
        """Rotates the waypoint counter-clockwise around the ship 90 degrees."""
        # Same as clockwise, in reverse. Translate (x, y) to (-y, x)
        self.waypoint_x, self.waypoint_y = -self.waypoint_y, self.waypoint_x

    def rotate_waypoint(self, direction: str, angle: int):
        """Rotates the waypoint around the ship in the given direction a number of times
        according to the angle provided.
        """
        turns = angle // 90
        if direction == self.ROTATE_COUNTERCLOCKWISE:
            rotate_func = self.rotate_waypoint_counterclockwise
        else:
            rotate_func = self.rotate_waypoint_clockwise
        
        for _ in range(turns):
            # For the number of times we need to rotate the waypoint 90 degrees,
            # run the chosen rotation function.
            rotate_func()

    def go_to_waypoint(self, times: int):
        """Move the ship towards the waypoint `times` number of times."""
        self.pos_x += (self.waypoint_x * times)
        self.pos_y += (self.waypoint_y * times)

    @property
    def manhattan_distance(self):
        return abs(self.pos_x) + abs(self.pos_y)

In [55]:
the_real_ferry = FerryPartDeux()
the_real_ferry.follow_directions(DIRECTIONS)

print("At the end of the directions,")
print(f"the REAL ferry is at position ({the_real_ferry.pos_x}, {the_real_ferry.pos_y}),")
print(f"with a Manhattan distance of {the_real_ferry.manhattan_distance}.")

At the end of the directions,
the ferry is at position (-11946, -16645),
with a Manhattan distance of 28591.
